In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go # or plotly.express as px 
from dash.dependencies import Input, Output
import numpy as np

#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__)

colors = {
    'background': '#FFFFFF',
    'text': '#000000'}

#data section
df = pd.read_csv("ALL_TWEET_SENTIMENT.csv")
df2 = pd.read_csv("unemployment.csv")

df['Count of China Virus'] = df['Text'].str.count('China Virus')
df['Count of fuckchina'] = df['Text'].str.count('fuckchina')
df['Count of Chinacoronavirus'] = df['Text'].str.count('Chinacoronavirus')
df['Count of China Corona Virus'] = df['Text'].str.count('China Corona Virus')
df['Count of Go Back to China'] = df['Text'].str.count('Go Back to China')
df['Count of chinaliedpeopledied'] = df['Text'].str.count('chinaliedpeopledied')
df['Count of Asian Virus'] = df['Text'].str.count('Asian Virus')
df['Count of Wuhan Virus'] = df['Text'].str.count('Wuhan Virus')
df['Count of Chinese Virus'] = df['Text'].str.count('Chinese Virus')
df['Count of Bat Eater'] = df['Text'].str.count('Bat Eater')
df['Count of nukechina'] = df['Text'].str.count('nukechina')
#app layout section
result = df.groupby(                                        \
      [pd.Grouper(key='Datetime', freq='1w'), 'Count of China Virus', 'count'] \
    ).sum().unstack(fill_value=0).stack().reset_index()

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    
    html.H1("Racist Tweet Growth Rate Since Covid Hit", 
            style={'textAlign': 'center',
                   'color': colors['text']}),
        dcc.Dropdown(id = "selected_keyword",
                options=[
                    {"label": x, "value": x} for x in sorted(df['key word'].unique())],
                 multi=False,
                value= "China Virus"
                 ), 
    html.Div(id ='output_container'),
    html.Br(),    
    dcc.Graph(id = 'my_tweet_map', figure={}),
    dcc.Graph(id = 'my_unemployment_map', figure={})
])

@app.callback(
     [Output(component_id='my_tweet_map', component_property='figure'),
       Output(component_id='my_unemployment_map', component_property='figure')],
    [Input(component_id='selected_keyword', component_property='value')])
def update_graph(option_selected): 
#     print(option_selected)
#     print(type(option_selected))
    y = str(option_selected)
    dff = result.copy() #Must use a copy of dataframe
    dff = dff[dff['key word']== option_selected]
    #dff = dff[dff['incident_type']== "Fire"]
#plotly express
    fig = px.line(dff, x="Datetime", y=dff['Count of'+' {}'.format(y)], title = "Covid Cases Increases by Date in Different States")
    fig2 = px.line(df2, x="Datetime", y= 'Unemployment_Rate ', title = "Covid Cases Increases by Date in Different States")
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black')

    return fig, fig2 #the return obj will be the output and if there are many output, it will go in order ( 1 obj => 1st output)


if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False, port = 9001)